# SimFin Test All Datasets

This Notebook performs automated testing of all the bulk datasets from SimFin. The datasets are first downloaded from the SimFin server and then various tests are performed on the data. An exception is raised if any problems are found.

This Notebook can be run as usual if you have `simfin` installed, by running the following command from the directory where this Notebook is located:

    jupyter notebook

This Notebook can also be run using `pytest` which makes automated testing easier. You need to have the Python packages `simfin` and `nbval` installed. Then execute the following command from the directory where this Notebook is located:

    pytest --nbval-lax test_bulk_data.ipynb
    
This runs the entire Notebook and outputs error messages for all the cells that raised an exception.

## Imports

In [1]:
import pandas as pd
import numpy as np
import sys

In [2]:
import simfin as sf
from simfin.names import *
from simfin.datasets import *

## Configure SimFin

In [3]:
sf.set_data_dir('~/simfin_data/')

In [4]:
sf.load_api_key(path='~/simfin_api_key.txt', default_key='free')

## Load All Datasets

In [5]:
%%time
# Use refresh_days=0 to always download new data.
data = AllDatasets(refresh_days=10)

Dataset "markets" is being downloaded ... 
- Download progress: 100.0%
- Extracting zip-file: Done!
Loading "ca-income-annual" from disk (0 days old).
Loading "de-income-annual" from disk (0 days old).
Loading "it-income-annual" from disk (0 days old).
Loading "sg-income-annual" from disk (0 days old).
Loading "us-income-annual" from disk (0 days old).
Loading "ca-income-annual-full" from disk (0 days old).
Loading "de-income-annual-full" from disk (0 days old).
Loading "it-income-annual-full" from disk (0 days old).
Loading "sg-income-annual-full" from disk (0 days old).
Loading "us-income-annual-full" from disk (0 days old).
Loading "ca-income-quarterly" from disk (0 days old).
Loading "de-income-quarterly" from disk (0 days old).
Loading "it-income-quarterly" from disk (0 days old).
Loading "sg-income-quarterly" from disk (0 days old).
Loading "us-income-quarterly" from disk (0 days old).
Loading "ca-income-quarterly-full" from disk (0 days old).
Loading "de-income-quarterly-full" f

Loading "ca-balance-banks-ttm" from disk (0 days old).
Loading "de-balance-banks-ttm" from disk (0 days old).
Loading "it-balance-banks-ttm" from disk (0 days old).
Loading "sg-balance-banks-ttm" from disk (0 days old).
Loading "us-balance-banks-ttm" from disk (0 days old).
Loading "ca-balance-banks-ttm-full" from disk (0 days old).
Loading "de-balance-banks-ttm-full" from disk (0 days old).
Loading "it-balance-banks-ttm-full" from disk (0 days old).
Loading "sg-balance-banks-ttm-full" from disk (0 days old).
Loading "us-balance-banks-ttm-full" from disk (0 days old).
Loading "ca-balance-insurance-annual" from disk (0 days old).
Loading "de-balance-insurance-annual" from disk (0 days old).
Loading "it-balance-insurance-annual" from disk (0 days old).
Loading "sg-balance-insurance-annual" from disk (0 days old).
Loading "us-balance-insurance-annual" from disk (0 days old).
Loading "ca-balance-insurance-annual-full" from disk (0 days old).
Loading "de-balance-insurance-annual-full" from 

Loading "ca-companies" from disk (0 days old).
Loading "de-companies" from disk (0 days old).
Loading "it-companies" from disk (0 days old).
Loading "sg-companies" from disk (0 days old).
Loading "us-companies" from disk (0 days old).
Loading "industries" from disk (0 days old).
Loading "markets" from disk (0 days old).
CPU times: user 9.44 s, sys: 871 ms, total: 10.3 s
Wall time: 10.4 s


In [6]:
# Example for annual Income Statements.
data.get(dataset='income', variant='annual', market='us').head()

,Ticker,SimFinId,Currency,Fiscal Year,Fiscal Period,Report Date,Publish Date,Shares (Basic),Shares (Diluted),Revenue,...,Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adjusted",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) From Continuing Operations,Net Extraordinary Gains (Losses),Net Income,Net Income Available to Common Shareholders
0,GOOG,18,USD,2016,FY,2016-12-31,2017-02-03,687778000.0,747565000.0,9.027200e+10,...,4.340000e+08,NaN,24150000000,NaN,24150000000,-4.672000e+09,19478000000,0.0,19478000000,19478000000
1,GOOG,18,USD,2015,FY,2015-12-31,2016-02-11,684626000.0,744675000.0,7.498900e+10,...,2.910000e+08,NaN,19651000000,NaN,19651000000,-3.303000e+09,16348000000,0.0,16348000000,15826000000
2,GOOG,18,USD,2014,FY,2014-12-31,2015-10-29,675935000.0,741998000.0,6.600100e+10,...,7.630000e+08,NaN,17259000000,NaN,17259000000,-3.639000e+09,13620000000,516000000.0,14136000000,14136000000
3,GOOG,18,USD,2013,FY,2013-12-31,2015-10-29,665692000.0,736950000.0,5.551900e+10,...,4.960000e+08,NaN,15899000000,NaN,15899000000,-2.739000e+09,13160000000,-427000000.0,12733000000,12733000000
4,GOOG,18,USD,2017,FY,2017-12-31,2018-02-06,692901000.0,750730000.0,1.108550e+11,...,1.047000e+09,NaN,29929000000,-2.736000e+09,27193000000,-1.453100e+10,12662000000,NaN,12662000000,12662000000


## Function for Testing Datasets

In [7]:
def test_datasets(test_name, datasets=None, variants=None,
                  markets=None,
                  test_func=None, test_func_rows=None,
                  process_df_none=False, raise_exception=True):
    """
    Helper-function for running tests on many Pandas DataFrames.
    
    :param test_name:
        String with the name of the test.
        
    :param datasets:
        By default (datasets=None) all possible datasets
        will be tested. Otherwise datasets is a list of
        strings with dataset names to be tested.
        
    :param variants:
        By default (variants=None) all possible variants
        for each dataset will be tested, as defined in
        simfin.datasets.valid_variants. Otherwise variants
        is a list of strings and only those variants
        will be tested.
        
    :param markets:
        By default (markets=None) all possible markets
        for each dataset will be tested, as defined in
        simfin.datasets.valid_markets. Otherwise markets
        is a list of strings and only those markets
        will be tested.
        
    :param test_func:
        Function to be called on the Pandas DataFrame for
        each dataset. If there are problems with the DataFrame
        then return True, otherwise return False.
        
        This is generally used for testing problems with the
        entire DataFrame. For example, if the dataset is empty:

        test_func = lambda df: len(df) == 0
        
        If this returns True then there is a problem with df.
                
    :param test_func_rows:
        Similar to test_func but for testing individual rows
        of a DataFrame. For example, test if SHARES_BASIC is
        None, zero or negative:
        
        test_func_rows = lambda df: (df[SHARES_BASIC] is None or
                                     df[SHARES_BASIC] <= 0)

    :param process_df_none:
        Boolean whether to process (True) or skip (False)
        DataFrames that are None, because they could not be loaded.

    :param raise_exception:
        Boolean. If True then raise an exception if there were
        any problems, but wait until all datasets have been
        tested, so we can print the list of datasets with problems.
        
    :return:
        None
    """
    
    # Convert test_func_rows to test_func.
    if test_func_rows is not None:
        test_func = lambda df: test_func_rows(df).any()

    # Number of problems found.
    num_problems = 0

    # For all datasets, variants and markets.
    for dataset, variant, market, df in data.iter(datasets=datasets,
                                                  variants=variants,
                                                  markets=markets):
        # Also process DataFrames that are None,
        # because they could not be loaded?
        if df is not None or process_df_none:
            try:
                # Perform the user-supplied test.
                problem_found = test_func(df)
            except:
                # An exception occurred so we consider
                # that to be a problem.
                problem_found = True
                
            if problem_found:
                # Increase the number of problems found.
                num_problems += 1

                # Print the test's name. Only done once.
                if num_problems==1:
                    print(test_name, file=sys.stderr)

                # Print the dataset details.
                msg = "dataset='{}', variant='{}', market='{}'"
                msg = msg.format(dataset, variant, market)
                print(msg, file=sys.stderr)
                
    # Raise exception?
    if num_problems>0 and raise_exception:
        raise Exception(test_name)

## Function for Getting Rows with Problems

When a test has found problems in a dataset, it does not show which specific rows have the problem. You can get all the problematic rows using this function:

In [8]:
def get_problem_rows(df, test_func_rows):
    """
    Perform the given test on all rows of the given DataFrame
    and return a DataFrame with only the problematic rows.
    
    :param df:
        Pandas DataFrame.

    :param test_func_rows:
        Function used for testing each row. This takes
        a Pandas DataFrame as an argument and returns
        a Pandas Series of booleans whether each row
        in the original DataFrame has the error.
        
        For example:
        
        test_func_rows = lambda df: (df[SHARES_BASIC] is None or
                                     df[SHARES_BASIC] <= 0)

    :return:
        Pandas DataFrame with only the problematic rows.
    """

    # Index of the rows with problems.
    idx = test_func_rows(df)
    
    # Extract the rows with problems.
    df2 = df[idx]
    
    return df2

## Function for Testing Equality with Tolerance

This function is useful when comparing floating point numbers, or when comparing accounting numbers that are supposed to have a strict relationship (e.g. Assets = Liabilities + Equity) but we might tolerate a small degree of error in the data e.g. 1%.

In [9]:
def isclose(x, y, tolerance=0.01):
    """
    Compare whether x and y are approximately equal within
    the given tolerance, which is a ratio so tolerance=0.01
    means that we tolerate max 1% difference between x and y.
    
    This is similar to numpy.isclose() but is a more efficient
    implementation for Pandas which apparently does not have
    this built-in already (v. 0.25.1)
    
    :param x:
        Pandas DataFrame or Series.

    :param y:
        Pandas DataFrame or Series.

    :param tolerance:
        Max allowed difference as a ratio e.g. 0.01 = 1%.

    :return:
        Pandas DataFrame or Series with booleans whether
        x and y are approx. equal.
    """
    return (x-y).abs() <= tolerance * y.abs()

# Tests

## Dataset could not be loaded

In [10]:
test_name = "Dataset could not be loaded"
test_func = lambda df: df is None
test_datasets(datasets=datasets_all,
              test_name=test_name, test_func=test_func,
              process_df_none=True)

## Dataset is empty

In [11]:
test_name = "Dataset is empty"
test_func = lambda df: len(df) == 0
test_datasets(datasets=datasets_all,
              test_name=test_name, test_func=test_func)

Dataset is empty
dataset='income', variant='annual', market='ca'
dataset='income', variant='annual', market='it'
dataset='income', variant='annual-full', market='ca'
dataset='income', variant='annual-full', market='it'
dataset='income', variant='quarterly', market='ca'
dataset='income', variant='quarterly', market='it'
dataset='income', variant='quarterly-full', market='ca'
dataset='income', variant='quarterly-full', market='it'
dataset='income', variant='ttm', market='ca'
dataset='income', variant='ttm', market='it'
dataset='income', variant='ttm-full', market='ca'
dataset='income', variant='ttm-full', market='it'
dataset='income-banks', variant='annual', market='ca'
dataset='income-banks', variant='annual', market='it'
dataset='income-banks', variant='annual', market='sg'
dataset='income-banks', variant='annual-full', market='ca'
dataset='income-banks', variant='annual-full', market='it'
dataset='income-banks', variant='annual-full', market='sg'
dataset='income-banks', variant='quart

Exception: Dataset is empty

In [12]:
data.get(dataset='shareprices', variant='daily', market='ca')

,Ticker,SimFinId,Date,Open,Low,High,Close,Adj. Close,Dividend,Volume


## Shares Basic is None or <= 0

In [17]:
test_name = "SHARES_BASIC is None or <= 0"
test_func_rows = lambda df: (df[SHARES_BASIC] is None or
                             df[SHARES_BASIC] <= 0)
test_datasets(datasets=datasets_fundamental,
              test_name=test_name, test_func_rows=test_func_rows)

SHARES_BASIC is None or <= 0
dataset='income', variant='annual', market='us'
dataset='income', variant='annual-full', market='us'
dataset='income', variant='quarterly', market='us'
dataset='income', variant='quarterly-full', market='us'
dataset='income', variant='ttm', market='us'
dataset='income', variant='ttm-full', market='us'
dataset='income-banks', variant='quarterly', market='us'
dataset='income-banks', variant='quarterly-full', market='us'
dataset='income-banks', variant='ttm', market='us'
dataset='income-banks', variant='ttm-full', market='us'
dataset='income-insurance', variant='quarterly', market='us'
dataset='income-insurance', variant='quarterly-full', market='us'
dataset='income-insurance', variant='ttm', market='us'
dataset='income-insurance', variant='ttm-full', market='us'
dataset='balance', variant='annual', market='us'
dataset='balance', variant='annual-full', market='us'
dataset='balance', variant='quarterly', market='us'
dataset='balance', variant='quarterly-full', 

Exception: SHARES_BASIC is None or <= 0

In [14]:
# Show the problematic rows for a dataset.
df = data.get(dataset='income', variant='annual', market='us')
get_problem_rows(df=df, test_func_rows=test_func_rows)

,Ticker,SimFinId,Currency,Fiscal Year,Fiscal Period,Report Date,Publish Date,Shares (Basic),Shares (Diluted),Revenue,...,Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adjusted",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) From Continuing Operations,Net Extraordinary Gains (Losses),Net Income,Net Income Available to Common Shareholders
1221,AAL,68568,USD,2010,FY,2010-12-31,2011-02-16,0.0,0.0,2.217000e+10,...,-8.140000e+08,-7.970000e+08,-506000000,0.0,-506000000,35000000.0,-471000000,NaN,-471000000,-471000000
1222,AAL,68568,USD,2009,FY,2009-12-31,2010-07-21,0.0,0.0,1.991700e+10,...,-7.480000e+08,-7.100000e+08,-1752000000,NaN,-1752000000,284000000.0,-1468000000,NaN,-1468000000,-1468000000
8144,SPB,357600,USD,2017,FY,2017-09-30,2017-11-17,0.0,0.0,5.007400e+09,...,-2.168000e+08,-2.111000e+08,426000000,-81400000.0,344600000,-47500000.0,297100000,NaN,295800000,295800000
8145,SPB,357600,USD,2016,FY,2016-09-30,2016-08-03,0.0,0.0,5.039700e+09,...,-2.586000e+08,-2.500000e+08,453700000,-56100000.0,397600000,-40000000.0,357600000,NaN,357100000,357100000
8146,SPB,357600,USD,2015,FY,2015-09-30,2015-08-06,0.0,0.0,4.690400e+09,...,-2.808000e+08,-2.719000e+08,278700000,-85400000.0,193300000,-43900000.0,149400000,NaN,148900000,148900000
8147,SPB,357600,USD,2014,FY,2014-09-30,2014-08-06,0.0,0.0,4.429100e+09,...,-2.084000e+08,-2.021000e+08,312800000,-39300000.0,273500000,-59000000.0,214500000,NaN,214100000,214100000
8148,SPB,357600,USD,2013,FY,2013-09-30,2013-08-08,0.0,0.0,4.085600e+09,...,-3.791000e+08,-3.756000e+08,20500000,-48400000.0,-27900000,-27400000.0,-55300000,NaN,-55200000,-55200000
8149,SPB,357600,USD,2012,FY,2012-09-30,2012-08-07,0.0,0.0,3.252435e+09,...,-1.927890e+08,-1.919110e+08,140023000,-31066000.0,108957000,-60385000.0,48572000,NaN,48572000,48572000
8150,SPB,357600,USD,2011,FY,2011-09-30,2011-08-11,0.0,0.0,3.154466e+09,...,-2.108200e+08,-2.083290e+08,53727000,-36603000.0,17124000,-92295000.0,-75171000,NaN,-75171000,-75171000
8151,SPB,357600,USD,2010,FY,2010-09-30,2011-08-11,0.0,0.0,2.567011e+09,...,-2.893150e+08,-2.770150e+08,-82085000,-42098000.0,-124183000,-63189000.0,-187372000,-2735000.0,-190107000,-190107000


## Shares Diluted is None or <= 0

In [18]:
test_name = "SHARES_DILUTED is None or <= 0"
test_func_rows = lambda df: (df[SHARES_DILUTED] is None or
                             df[SHARES_DILUTED] <= 0)
test_datasets(datasets=datasets_fundamental,
              test_name=test_name, test_func_rows=test_func_rows)

SHARES_DILUTED is None or <= 0
dataset='income', variant='annual', market='us'
dataset='income', variant='annual-full', market='us'
dataset='income', variant='quarterly', market='us'
dataset='income', variant='quarterly-full', market='us'
dataset='income', variant='ttm', market='us'
dataset='income', variant='ttm-full', market='us'
dataset='income-banks', variant='quarterly', market='us'
dataset='income-banks', variant='quarterly-full', market='us'
dataset='income-banks', variant='ttm', market='us'
dataset='income-banks', variant='ttm-full', market='us'
dataset='income-insurance', variant='quarterly', market='us'
dataset='income-insurance', variant='quarterly-full', market='us'
dataset='income-insurance', variant='ttm', market='us'
dataset='income-insurance', variant='ttm-full', market='us'
dataset='balance', variant='annual', market='us'
dataset='balance', variant='annual-full', market='us'
dataset='balance', variant='quarterly', market='us'
dataset='balance', variant='quarterly-full'

Exception: SHARES_DILUTED is None or <= 0

In [19]:
# Show the problematic rows for a dataset.
df = data.get(dataset='income', variant='quarterly', market='us')
get_problem_rows(df=df, test_func_rows=test_func_rows)

,Ticker,SimFinId,Currency,Fiscal Year,Fiscal Period,Report Date,Publish Date,Shares (Basic),Shares (Diluted),Revenue,...,Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adjusted",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) From Continuing Operations,Net Extraordinary Gains (Losses),Net Income,Net Income Available to Common Shareholders
612,ENS,35886,USD,2010,Q4,2011-03-31,2011-02-09,-1.248976e+09,-1.266011e+09,4.505370e+08,...,-6217000.0,-5991000.0,30983000,-6163000.0,24820000,-6991000.0,17829000,NaN,17829000,17829000
613,ENS,35886,USD,2011,Q4,2012-03-31,2011-05-31,-3.893758e+08,-3.943303e+08,5.480540e+08,...,-5907000.0,-4361000.0,49197000,-9741000.0,39456000,-9365000.0,30091000,NaN,30091000,30091000
819,JOY,39382,USD,2009,Q4,2009-10-31,2010-06-08,-3.068902e+08,-3.085706e+08,9.635280e+08,...,-5322000.0,-5322000.0,178780000,5585000.0,184365000,-60340000.0,124025000,NaN,124025000,124025000
1008,AXON,41996,USD,2018,Q4,2019-03-31,2019-06-11,-2.930225e+08,-2.930225e+08,NaN,...,-1447000.0,-1722000.0,-14753000,NaN,-9137000,91000.0,-9046000,NaN,-9046000,-9046000
1383,A,45846,USD,2013,Q4,2013-10-31,2013-12-19,-7.940000e+07,-8.160000e+07,-1.170000e+09,...,-32000000.0,-28000000.0,-312000000,NaN,-312000000,24000000.0,-288000000,NaN,221000000,221000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64597,JAKK,908372,USD,2016,Q4,2016-12-31,2017-03-16,1.650400e+07,-6.464000e+06,1.670260e+08,...,-3453000.0,-3504000.0,-5844000,NaN,-5844000,-1908000.0,-7752000,NaN,-7585000,-7585000
64779,KYAK,910478,USD,2011,Q1,2011-03-31,2012-09-20,7.397969e+06,-2.248806e+07,5.267400e+07,...,632000.0,21000.0,3587000,-14980000.0,-11393000,4479000.0,-6914000,NaN,-6914000,-9850000
64890,BWXT,914354,USD,2013,Q4,2013-12-31,2014-02-26,-2.409151e+07,-2.406707e+07,-9.197300e+08,...,-24216000.0,-1335000.0,37972000,5725000.0,43697000,-30582000.0,13115000,NaN,165588000,165588000
64898,BWXT,914354,USD,2011,Q4,2011-12-31,2012-05-09,-1.645524e+11,-1.656942e+11,8.007890e+08,...,18330000.0,-1690000.0,-54552000,3713000.0,-50839000,21544000.0,-29295000,NaN,-27063000,-27063000


## Revenue is negative

In [20]:
test_name = "REVENUE < 0"
test_func_rows = lambda df: (df[REVENUE] < 0)
test_datasets(datasets=datasets_income,
              test_name=test_name, test_func_rows=test_func_rows)

REVENUE < 0
dataset='income', variant='annual', market='us'
dataset='income', variant='annual-full', market='us'
dataset='income', variant='quarterly', market='us'
dataset='income', variant='quarterly-full', market='us'
dataset='income', variant='ttm', market='us'
dataset='income', variant='ttm-full', market='us'
dataset='income-banks', variant='annual', market='ca'
dataset='income-banks', variant='annual', market='de'
dataset='income-banks', variant='annual', market='it'
dataset='income-banks', variant='annual', market='sg'
dataset='income-banks', variant='annual', market='us'
dataset='income-banks', variant='annual-full', market='ca'
dataset='income-banks', variant='annual-full', market='de'
dataset='income-banks', variant='annual-full', market='it'
dataset='income-banks', variant='annual-full', market='sg'
dataset='income-banks', variant='annual-full', market='us'
dataset='income-banks', variant='quarterly', market='ca'
dataset='income-banks', variant='quarterly', market='de'
datase

Exception: REVENUE < 0

In [21]:
# Show the problematic rows for a dataset.
df = data.get(dataset='income', variant='quarterly', market='us')
get_problem_rows(df=df, test_func_rows=test_func_rows)

,Ticker,SimFinId,Currency,Fiscal Year,Fiscal Period,Report Date,Publish Date,Shares (Basic),Shares (Diluted),Revenue,...,Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adjusted",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) From Continuing Operations,Net Extraordinary Gains (Losses),Net Income,Net Income Available to Common Shareholders
59,FOX,242,USD,2011,Q4,2011-06-30,2011-08-15,2.626000e+09,2.647000e+09,-2.110000e+08,...,-175000000.0,-266000000.0,278000000,-3000000.0,275000000,-16000000.0,259000000,NaN,683000000,683000000
60,FOX,242,USD,2012,Q4,2012-06-30,2012-08-14,2.413000e+09,2.414000e+09,-2.850000e+08,...,-101000000.0,-273000000.0,663000000,-47000000.0,616000000,-163000000.0,453000000,NaN,-1553000000,-1553000000
276,PPL,26853,USD,2013,Q4,2013-12-31,2014-02-24,5.061656e+08,5.272955e+08,-1.718000e+09,...,-98000000.0,-25000000.0,166000000,NaN,166000000,-19000000.0,147000000,NaN,-98000000,-98000000
1383,A,45846,USD,2013,Q4,2013-10-31,2013-12-19,-7.940000e+07,-8.160000e+07,-1.170000e+09,...,-32000000.0,-28000000.0,-312000000,NaN,-312000000,24000000.0,-288000000,NaN,221000000,221000000
1510,HPE,47023,USD,2015,Q4,2015-10-31,2016-09-08,-2.069368e+09,3.146594e+09,-7.582000e+09,...,33000000.0,35000000.0,401000000,174000000.0,575000000,989000000.0,1564000000,NaN,1385000000,1385000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65960,MSB,942591,USD,2018,Q4,2019-01-31,2019-04-12,1.312001e+07,NaN,-3.954987e+07,...,NaN,NaN,6786679,NaN,6786679,NaN,6786679,NaN,6786679,6786679
65962,MSB,942591,USD,2010,Q4,2011-01-31,2012-04-13,NaN,NaN,-2.778889e+07,...,NaN,NaN,5272369,NaN,5272369,NaN,5272369,NaN,5272369,5272369
65974,INVA,942592,USD,2014,Q1,2014-03-31,2014-05-07,1.098590e+08,1.098590e+08,-7.800000e+05,...,-1459000.0,-1644000.0,-16182000,NaN,-16182000,NaN,-16182000,-51521000.0,-67703000,-67703000
65993,INVA,942592,USD,2012,Q4,2012-12-31,2013-02-26,3.158136e+07,2.105376e+07,-1.243460e+08,...,-1344000.0,-1500000.0,-43471000,NaN,-43471000,NaN,-43471000,NaN,-31324000,-31324000


## Assets != Liabilities + Equity (Exact Comparison)

In [22]:
test_name = "Assets != Liabilities + Equity (Exact Comparison)"
test_func_rows = lambda df: (df[TOTAL_ASSETS] != df[TOTAL_LIABILITIES] + df[TOTAL_EQUITY])
test_datasets(datasets=datasets_balance,
              test_name=test_name, test_func_rows=test_func_rows)

Assets != Liabilities + Equity (Exact Comparison)
dataset='balance', variant='annual', market='de'
dataset='balance', variant='annual', market='us'
dataset='balance', variant='annual-full', market='de'
dataset='balance', variant='annual-full', market='us'
dataset='balance', variant='quarterly', market='de'
dataset='balance', variant='quarterly', market='us'
dataset='balance', variant='quarterly-full', market='de'
dataset='balance', variant='quarterly-full', market='us'
dataset='balance', variant='ttm', market='de'
dataset='balance', variant='ttm', market='us'
dataset='balance', variant='ttm-full', market='de'
dataset='balance', variant='ttm-full', market='us'
dataset='balance-banks', variant='annual', market='us'
dataset='balance-banks', variant='annual-full', market='us'
dataset='balance-banks', variant='quarterly', market='de'
dataset='balance-banks', variant='quarterly', market='us'
dataset='balance-banks', variant='quarterly-full', market='de'
dataset='balance-banks', variant='quar

Exception: Assets != Liabilities + Equity (Exact Comparison)

In [23]:
# Get the problematic rows for a dataset.
df = data.get(dataset='balance', variant='quarterly', market='us')
df2 = get_problem_rows(df=df, test_func_rows=test_func_rows)

# Only show the relevant columns.
df2[[TICKER, SIMFIN_ID, REPORT_DATE, TOTAL_ASSETS, TOTAL_LIABILITIES, TOTAL_EQUITY]]

,Ticker,SimFinId,Report Date,Total Assets,Total Liabilities,Total Equity
121,BLK,914,2011-02-28,178459000000,1.521250e+11,2.633300e+10
124,BLK,914,2010-02-28,178124000000,1.535220e+11,2.460100e+10
398,ACAT,33956,2011-09-30,357162000,1.522490e+08,2.049120e+08
494,ENS,35886,2016-09-30,2264838000,1.185643e+09,1.076018e+09
495,ENS,35886,2016-06-30,2204571000,1.168335e+09,1.030288e+09
...,...,...,...,...,...,...
61270,BLGO,943380,2013-12-31,126196,7.321570e+05,-5.649640e+05
61271,BLGO,943380,2013-09-30,94249,5.013070e+05,-3.633310e+05
61272,BLGO,943380,2013-06-30,127168,7.481080e+05,-5.744830e+05
61273,BLGO,943380,2013-03-31,151457,6.816830e+05,-4.810390e+05


## Assets != Liabilities + Equity (1% Tolerance)

The above test used exact comparison. We now allow for 1% error.

In [24]:
def test_func_rows(df):
    x = df[TOTAL_ASSETS]
    y = df[TOTAL_LIABILITIES] + df[TOTAL_EQUITY]
    
    # Compare x and y within 1% tolerance. Note the resulting
    # boolean array is negated because we want to indicate
    # which rows are problematic so x and y are not close.
    return ~isclose(x=x, y=y, tolerance=0.01)

In [25]:
test_name = "Assets != Liabilities + Equity (1% Tolerance)"
test_datasets(datasets=datasets_balance,
              test_name=test_name, test_func_rows=test_func_rows)

Assets != Liabilities + Equity (1% Tolerance)
dataset='balance', variant='annual', market='de'
dataset='balance', variant='annual', market='us'
dataset='balance', variant='annual-full', market='de'
dataset='balance', variant='annual-full', market='us'
dataset='balance', variant='quarterly', market='de'
dataset='balance', variant='quarterly', market='us'
dataset='balance', variant='quarterly-full', market='de'
dataset='balance', variant='quarterly-full', market='us'
dataset='balance', variant='ttm', market='de'
dataset='balance', variant='ttm', market='us'
dataset='balance', variant='ttm-full', market='de'
dataset='balance', variant='ttm-full', market='us'


Exception: Assets != Liabilities + Equity (1% Tolerance)

In [26]:
# Get the problematic rows for a dataset.
df = data.get(dataset='balance', variant='quarterly', market='us')
df2 = get_problem_rows(df=df, test_func_rows=test_func_rows)

# Only show the relevant columns.
df2[[TICKER, SIMFIN_ID, REPORT_DATE, TOTAL_ASSETS, TOTAL_LIABILITIES, TOTAL_EQUITY]]

,Ticker,SimFinId,Report Date,Total Assets,Total Liabilities,Total Equity
3034,AES,62329,2018-06-30,32597000000,2.607700e+10,5.657000e+09
3035,AES,62329,2018-03-31,32573000000,2.619700e+10,5.525000e+09
3036,AES,62329,2017-12-31,33112000000,2.743000e+10,4.845000e+09
3037,AES,62329,2017-09-30,38834000000,3.168800e+10,6.179000e+09
3038,AES,62329,2017-06-30,36469000000,2.960400e+10,6.074000e+09
...,...,...,...,...,...,...
61270,BLGO,943380,2013-12-31,126196,7.321570e+05,-5.649640e+05
61271,BLGO,943380,2013-09-30,94249,5.013070e+05,-3.633310e+05
61272,BLGO,943380,2013-06-30,127168,7.481080e+05,-5.744830e+05
61273,BLGO,943380,2013-03-31,151457,6.816830e+05,-4.810390e+05


## Dates are invalid (Fundamentals)

In [27]:
# Lambda function for converting strings to dates. Format: YYYY-MM-DD
# This will raise an exception if invalid dates are encountered.
date_parser = lambda column: pd.to_datetime(column, yearfirst=True, dayfirst=False)

In [28]:
# Test function for the entire DataFrame.
# This cannot show which individual rows have problems.
def test_func(df):
    result1 = date_parser(df[REPORT_DATE])
    result2 = date_parser(df[PUBLISH_DATE])
    
    # We only get to this point if date_parser() does not
    # raise any exceptions, in which case we assume the
    # data did not have any problems.
    return False

In [29]:
test_name = "REPORT_DATE or PUBLISH_DATE is invalid"
test_datasets(datasets=datasets_fundamental,
              test_name=test_name, test_func=test_func)

## Dates are invalid (Share-Prices)

In [30]:
# Test function for the entire DataFrame.
# This cannot show which individual rows have problems.
def test_func(df):
    result1 = date_parser(df[DATE])
    
    # We only get to this point if date_parser() does not
    # raise any exceptions, in which case we assume the
    # data did not have any problems.
    return False

In [31]:
test_name = "DATE is invalid"
test_datasets(datasets=datasets_shareprices,
              test_name=test_name, test_func=test_func)

## Duplicate Tickers

In [32]:
def get_duplicate_tickers(df):
    """
    Return the rows of `df` where multiple SIMFIN_ID
    have the same TICKER.
    
    :param df: Pandas DataFrame with TICKER column.
    :return: Pandas DataFrame.
    """

    # Remove duplicate rows of [TICKER, SIMFIN_ID] pairs.
    # For the 'companies' dataset this is not necessary,
    # but for e.g. the 'income' dataset we have many rows
    # for each [TICKER, SIMFIN_ID] pair because there are
    # many financial reports for each of these ID pairs.
    idx = df[[TICKER, SIMFIN_ID]].duplicated()
    df2 = df[~idx]

    # Now the DataFrame df2 only contains unique rows of
    # [TICKER, SIMFIN_ID] so we need to check if there are
    # any duplicate TICKER.

    # Index for rows where TICKER is a duplicate.
    idx1 = df2[TICKER].duplicated()

    # Index for rows where TICKER is not NaN.
    # These would otherwise show up as duplicates.
    idx2 = df2[TICKER].notna()

    # Index for rows where TICKER is a duplicate but not NaN.
    idx = idx1 & idx2

    # Get those rows from the DataFrame.
    df2 = df2[idx]

    return df2

In [33]:
# Test-function whether a DataFrame has duplicate tickers.
test_func = lambda df: (len(get_duplicate_tickers(df=df)) > 0)

In [34]:
# Test whether these datasets have duplicate tickers.
# It might only be necessary for 'companies' but we
# might as well test all datasets that use tickers.
_datasets = ['companies'] + datasets_fundamental + datasets_shareprices

In [35]:
test_name = "Duplicate Tickers"
test_datasets(datasets=_datasets,
              test_name=test_name, test_func=test_func)

Duplicate Tickers
dataset='companies', variant='None', market='us'
dataset='income', variant='annual', market='us'
dataset='income', variant='annual-full', market='us'
dataset='income', variant='quarterly', market='us'
dataset='income', variant='quarterly-full', market='us'
dataset='income', variant='ttm', market='us'
dataset='income', variant='ttm-full', market='us'
dataset='balance', variant='annual', market='us'
dataset='balance', variant='annual-full', market='us'
dataset='balance', variant='quarterly', market='us'
dataset='balance', variant='quarterly-full', market='us'
dataset='balance', variant='ttm', market='us'
dataset='balance', variant='ttm-full', market='us'
dataset='cashflow', variant='annual', market='us'
dataset='cashflow', variant='annual-full', market='us'
dataset='cashflow', variant='quarterly', market='us'
dataset='cashflow', variant='quarterly-full', market='us'
dataset='cashflow', variant='ttm', market='us'
dataset='cashflow', variant='ttm-full', market='us'
datase

Exception: Duplicate Tickers

In [36]:
# Show duplicate tickers in the 'companies' dataset.
df = data.get(dataset='companies', market='us')
get_duplicate_tickers(df=df)

,Ticker,SimFinId,Company Name,IndustryId
109,GOOG,61595,Google Inc.,101002.0
1097,MDT,378189,Medtronic INC,106004.0
1099,MYL,378191,Mylan N.V.,106005.0
1202,AGN,442340,Allergan Inc.,106005.0
1206,GGP,442374,"General Growth Properties, Inc",109001.0
1247,ES,446632,Energysolutions Inc,105001.0
1268,FE,447804,Metropolitan Edison Co,105001.0
1287,SHO,448662,Sunstone Hotel Investors Inc,109001.0
1298,WFT,449193,Weatherford International LTD,107004.0
1399,ENDP,530918,Endo International plc,106005.0


In [37]:
# Show duplicate tickers in the 'income-annual' dataset.
df = data.get(dataset='income', variant='annual', market='us')
get_duplicate_tickers(df=df)

,Ticker,SimFinId,Currency,Fiscal Year,Fiscal Period,Report Date,Publish Date,Shares (Basic),Shares (Diluted),Revenue,...,Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adjusted",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) From Continuing Operations,Net Extraordinary Gains (Losses),Net Income,Net Income Available to Common Shareholders
886,GOOG,61595,USD,2015,FY,2015-12-31,2016-02-11,6.846260e+08,7.446750e+08,7.498900e+10,...,2.910000e+08,NaN,19651000000,NaN,19651000000,-3.303000e+09,16348000000,0.0,16348000000,15826000000
9465,MDT,378189,USD,2013,FY,2013-04-30,2014-06-20,1.002100e+09,1.013600e+09,1.700500e+10,...,-2.890000e+08,-1.080000e+08,4710000000,-1.005000e+09,3705000000,-6.400000e+08,3065000000,0.0,3065000000,3065000000
9484,MYL,378191,USD,2014,FY,2014-12-31,2015-03-02,3.737000e+08,3.980000e+08,7.719600e+09,...,-3.781000e+08,-3.332000e+08,942400000,3.210000e+07,974500000,-4.140000e+07,933100000,NaN,929400000,929400000
10416,AGN,442340,USD,2014,FY,2014-12-31,2015-02-19,2.976000e+08,3.040000e+08,7.237900e+09,...,-2.000000e+07,-6.170000e+07,1989300000,0.000000e+00,1989300000,-4.567000e+08,1532600000,-3800000.0,1524200000,1524200000
10805,ES,446632,USD,2013,FY,2013-12-31,2014-03-31,NaN,NaN,1.804398e+09,...,-7.387500e+07,-7.677400e+07,-46896000,0.000000e+00,-46896000,-7.769000e+06,-54665000,NaN,-54653000,-54653000
10950,FE,447804,USD,2011,FY,2011-12-31,2012-02-28,3.990000e+08,4.010000e+08,1.625800e+10,...,-8.940000e+08,-1.008000e+09,1724000000,-2.810000e+08,1443000000,-5.740000e+08,869000000,NaN,885000000,885000000
11108,SHO,448662,USD,2015,FY,2015-12-31,2016-02-23,2.073500e+08,2.073500e+08,1.249180e+09,...,-6.263100e+07,-6.651600e+07,252868000,8.819000e+07,341058000,-1.434000e+06,339624000,15895000.0,355519000,355519000
11192,WFT,449193,USD,2013,FY,2013-12-31,2014-02-25,7.720000e+08,7.720000e+08,1.526300e+10,...,-5.930000e+08,-5.160000e+08,59000000,-2.290000e+08,-170000000,-1.440000e+08,-314000000,NaN,-345000000,-345000000
11987,ENDP,530918,USD,2017,FY,2017-12-31,2018-02-27,2.231980e+08,2.231980e+08,3.468858e+09,...,-4.712050e+08,-4.882280e+08,-32818000,-1.450186e+09,-1483004000,2.502930e+08,-1232711000,-802722000.0,-2035433000,-2035433000
12988,FTI,637893,USD,2017,FY,2017-12-31,2018-04-02,4.667000e+08,4.683000e+08,1.505690e+10,...,-2.855000e+08,-3.152000e+08,1110200000,-4.305000e+08,679700000,-5.455000e+08,134200000,NaN,113300000,113300000


## Missing Annual Reports

In [38]:
def missing_annual_reports(df):
    """
    Return a list of the SIMFIN_ID's from the given DataFrame
    that have missing annual reports.
    
    :param df:
        Pandas DataFrame with a dataset e.g. 'income-annual'.
        It must have columns SIMFIN_ID and FISCAL_YEAR.

    :return:
        List of integers with SIMFIN_ID's that have missing reports.
    """
    
    # The idea is to test for each SIMFIN_ID individually,
    # whether the DataFrame has all the expected reports for
    # consecutive Fiscal Years between the min/max years.
    
    # Helper-function for processing a DataFrame for one SIMFIN_ID.
    def _missing(df):
        # Get the Fiscal Years from the DataFrame.
        fiscal_years = df[FISCAL_YEAR]

        # How many years between min and max fiscal years.
        num_years = fiscal_years.max() - fiscal_years.min() + 1

        # We expect the Series to have the same length, otherwise
        # some reports must be missing between min and max years.
        missing = (num_years != len(fiscal_years))

        return missing
    
    # Process all companies individually and get a Pandas
    # DataFrame with a boolean for each SIMFIN_ID whether
    # it has some missing Fiscal Years.
    idx = df.groupby(SIMFIN_ID).apply(_missing)

    # List of the SIMFIN_ID's that have missing reports.
    simfin_ids = list(idx[idx].index.values)

    return simfin_ids

In [39]:
test_name = "Missing annual reports"
test_func = lambda df: len(missing_annual_reports(df=df)) > 0
test_datasets(datasets=datasets_fundamental,
              variants=['annual'],
              test_name=test_name, test_func=test_func)

Missing annual reports
dataset='income', variant='annual', market='de'
dataset='income', variant='annual', market='sg'
dataset='income', variant='annual', market='us'
dataset='income-insurance', variant='annual', market='us'
dataset='balance', variant='annual', market='de'
dataset='balance', variant='annual', market='us'
dataset='balance-banks', variant='annual', market='de'
dataset='balance-banks', variant='annual', market='us'
dataset='balance-insurance', variant='annual', market='us'
dataset='cashflow', variant='annual', market='us'
dataset='cashflow-banks', variant='annual', market='de'
dataset='cashflow-banks', variant='annual', market='us'
dataset='cashflow-insurance', variant='annual', market='us'


Exception: Missing annual reports

In [40]:
# Get list of SIMFIN_ID's that have missing reports for a dataset.
df = data.get(dataset='income', variant='annual', market='us')
missing_annual_reports(df=df)

[37910,
 67201,
 89750,
 117690,
 123505,
 133962,
 181659,
 257294,
 336622,
 337852,
 350340,
 353395,
 378142,
 410846,
 419436,
 446592,
 449198,
 511749,
 515845,
 592569,
 609619,
 625536,
 628030,
 628033,
 630304,
 646016,
 651624,
 652480,
 652699,
 653156,
 654363,
 658373,
 658466,
 659172,
 660712,
 662524,
 662568,
 664571,
 679470,
 679473,
 687925,
 689588,
 689895,
 703911,
 705165,
 705845,
 708961,
 709706,
 727266,
 733746,
 767563,
 795710,
 798466,
 815635,
 818918,
 819013,
 819348,
 819902,
 821235,
 821503,
 823898,
 824527,
 824887,
 877522,
 889944,
 890308,
 899359,
 900135,
 901127,
 905198,
 910562,
 914944,
 919132,
 923419]

In [41]:
def sort_annual_reports(df, simfin_id):
    """
    Get the data for a given SIMFIN_ID and set the index to be
    the sorted Fiscal Year so it is easier to see which are missing.
    """
    return df.set_index([SIMFIN_ID, FISCAL_YEAR]).sort_index().loc[simfin_id]

In [42]:
# Show all the reports for a given SIMFIN_ID sorted by
# Fiscal Year so it is easier to see which are missing.
sort_annual_reports(df=df, simfin_id=37910)

,Ticker,Currency,Fiscal Period,Report Date,Publish Date,Shares (Basic),Shares (Diluted),Revenue,Cost of Revenue,Gross Profit,...,Non-Operating Income (Loss),"Interest Expense, Net","Pretax Income (Loss), Adjusted",Abnormal Gains (Losses),Pretax Income (Loss),"Income Tax (Expense) Benefit, Net",Income (Loss) From Continuing Operations,Net Extraordinary Gains (Losses),Net Income,Net Income Available to Common Shareholders
Fiscal Year,,,,,,,,,,,,,,,,,,,,,
2012,CK00015847,USD,FY,2012-12-31,2015-03-26,NaN,NaN,2.365339e+09,-1.700756e+09,664583000.0,...,-58893000.0,-58893000.0,330289000,-351831000.0,-21542000,-1408000.0,-22950000,NaN,-22950000,-22950000
2014,CK00015847,USD,FY,2014-12-31,2015-03-26,NaN,NaN,2.508315e+09,-1.801798e+09,706517000.0,...,-83379000.0,-83379000.0,323574000,-345764000.0,-22190000,7523000.0,-14667000,NaN,-14667000,-14667000
2015,CK00015847,USD,FY,2015-12-31,2016-03-23,NaN,NaN,2.471949e+09,-1.743660e+09,728289000.0,...,-84053000.0,-84053000.0,326540000,-386805000.0,-60265000,-1978000.0,-62243000,NaN,-62243000,-62243000
2016,CK00015847,USD,FY,2016-12-31,2017-03-30,NaN,NaN,2.456767e+09,-1.730774e+09,725993000.0,...,-80889000.0,-80889000.0,324562000,-366913000.0,-42351000,-2212000.0,-44563000,NaN,-44563000,-44563000


## Missing Quarterly Reports

In [43]:
def missing_quarterly_reports(df):
    """
    Return a list of the SIMFIN_ID's from the given DataFrame
    that have missing quarterly or ttm reports.
    
    :param df:
        Pandas DataFrame with a dataset e.g. 'income-annual'.
        It must have columns SIMFIN_ID, FISCAL_YEAR, FISCAL_PERIOD.

    :return:
        List of integers with SIMFIN_ID's that have missing reports.
    """
    
    # The idea is to test for each SIMFIN_ID individually,
    # whether the DataFrame has all the expected reports for
    # consecutive Fiscal Years and Periods between the min/max.
    
    # Helper-function for processing a DataFrame for one SIMFIN_ID.
    def _missing(df):
        # Get the Fiscal Years and Periods from the DataFrame.
        fiscal_years_periods = df[[FISCAL_YEAR, FISCAL_PERIOD]]

        # The first Fiscal Year and Period.
        min_year = fiscal_years_periods[FISCAL_YEAR].min()
        min_idx = (fiscal_years_periods[FISCAL_YEAR] == min_year)
        min_period = fiscal_years_periods[min_idx][FISCAL_PERIOD].min()

        # The last Fiscal Year and Period.
        max_year = fiscal_years_periods[FISCAL_YEAR].max()
        max_idx = (fiscal_years_periods[FISCAL_YEAR] == max_year)
        max_period = fiscal_years_periods[max_idx][FISCAL_PERIOD].max()

        # How many years between min and max fiscal years.
        num_years = max_year - min_year + 1

        # Total number of Fiscal Periods between first and
        # last Fiscal Years - if all Fiscal Periods were included.
        num_periods = num_years * 4

        # Used to map from Fiscal Period strings to ints.
        # This is safer and easier to understand than
        # e.g. def map_period(x): int(x[1])
        map_period = \
        {
            'Q1': 1,
            'Q2': 2,
            'Q3': 3,
            'Q4': 4
        }

        # Number of Fiscal Periods missing in the first year.
        adj_min_period = map_period[min_period] - 1

        # Number of Fiscal Periods missing in the last year.
        adj_max_period = 4 - map_period[max_period]

        # Adjust the number of Fiscal Periods between the min/max
        # Fiscal Years and Periods by subtracting those periods
        # missing in the first and last years.
        expected_periods = num_periods - adj_min_period - adj_max_period

        # If the expected number of Fiscal Periods between the
        # min and max dates, is different from the actual number
        # of Fiscal Periods in the DataFrame, then some are missing.
        missing = (expected_periods != len(fiscal_years_periods))

        return missing

    # Process all companies individually and get a Pandas
    # DataFrame with a boolean for each SIMFIN_ID whether
    # it has some missing Fiscal Years.
    idx = df.groupby(SIMFIN_ID).apply(_missing)

    # List of the SIMFIN_ID's that have missing reports.
    simfin_ids = list(idx[idx].index.values)

    return simfin_ids

In [44]:
test_name = "Missing quarterly reports"
test_func = lambda df: len(missing_quarterly_reports(df=df)) > 0
test_datasets(datasets=datasets_fundamental,
              variants=['quarterly'],
              test_name=test_name, test_func=test_func)

Missing quarterly reports
dataset='income', variant='quarterly', market='de'
dataset='income', variant='quarterly', market='sg'
dataset='income', variant='quarterly', market='us'
dataset='income-banks', variant='quarterly', market='us'
dataset='income-insurance', variant='quarterly', market='us'
dataset='balance', variant='quarterly', market='de'
dataset='balance', variant='quarterly', market='us'
dataset='balance-banks', variant='quarterly', market='de'
dataset='balance-banks', variant='quarterly', market='us'
dataset='balance-insurance', variant='quarterly', market='us'
dataset='cashflow', variant='quarterly', market='de'
dataset='cashflow', variant='quarterly', market='us'
dataset='cashflow-banks', variant='quarterly', market='de'
dataset='cashflow-banks', variant='quarterly', market='us'
dataset='cashflow-insurance', variant='quarterly', market='us'


Exception: Missing quarterly reports

In [45]:
# Get list of SIMFIN_ID's that have missing reports for a dataset.
df = data.get(dataset='income', variant='quarterly', market='us')
missing_quarterly_reports(df=df)

[36205,
 60906,
 67201,
 73420,
 82671,
 82753,
 82963,
 86808,
 89750,
 91899,
 98663,
 108139,
 117059,
 117690,
 122208,
 122759,
 123505,
 123623,
 124551,
 133865,
 133962,
 136247,
 138152,
 139560,
 144111,
 164920,
 181659,
 184064,
 185450,
 188922,
 189248,
 210709,
 217619,
 230667,
 231124,
 233893,
 242378,
 243720,
 256896,
 257294,
 258154,
 258455,
 259098,
 286194,
 286288,
 286372,
 329205,
 333440,
 334921,
 336622,
 337852,
 340222,
 340525,
 350160,
 350340,
 352171,
 353395,
 353811,
 354347,
 358146,
 359467,
 360213,
 362372,
 368990,
 370817,
 378117,
 378121,
 378130,
 378142,
 378175,
 378179,
 378185,
 378197,
 378218,
 378225,
 410846,
 419436,
 436223,
 445830,
 446592,
 447804,
 448011,
 448043,
 448306,
 448662,
 448754,
 449068,
 449233,
 466098,
 495816,
 497906,
 505724,
 511510,
 511749,
 515845,
 543421,
 546306,
 546690,
 548204,
 549638,
 564852,
 569369,
 592569,
 606334,
 609619,
 617429,
 625536,
 626019,
 627775,
 628030,
 628032,
 628033,
 63

In [46]:
def sort_quarterly_reports(df, simfin_id):
    """
    Get the data for a given SIMFIN_ID and set the index to be
    the sorted Fiscal Year and Period so it is easier to see
    which ones are missing.
    """
    return df.set_index([SIMFIN_ID, FISCAL_YEAR, FISCAL_PERIOD]).sort_index().loc[simfin_id]

In [47]:
# Show all the reports for a given SIMFIN_ID sorted by
# Fiscal Year and Period so it is easier to see which are missing.
sort_quarterly_reports(df=df, simfin_id=139560)

Ticker Currency Report Date Publish Date  \
Fiscal Year Fiscal Period                                            
2012        Q1              REXR      USD  2012-03-31   2013-10-03   
            Q2              REXR      USD  2012-06-30   2013-10-03   
2013        Q1              REXR      USD  2013-03-31   2013-10-03   
            Q2              REXR      USD  2013-06-30   2013-10-03   
            Q3              REXR      USD  2013-09-30   2013-11-13   
2014        Q1              REXR      USD  2014-03-31   2014-05-14   
            Q2              REXR      USD  2014-06-30   2014-08-11   
            Q3              REXR      USD  2014-09-30   2014-11-06   
            Q4              REXR      USD  2014-12-31   2015-03-09   
2015        Q1              REXR      USD  2015-03-31   2015-05-11   
            Q2              REXR      USD  2015-06-30   2015-08-10   
            Q3              REXR      USD  2015-09-30   2015-11-09   
            Q4              REXR      USD  2015-12-31   2016-02-25   
2016        Q1              REXR      USD  2016-03-31   2016-05-10   
            Q2              REXR      USD  2016-06-30   2016-08-08   
            Q3              REXR      USD  2016-09-30   2016-11-07   
            Q4              REXR      USD  2016-12-31   2017-02-23   
2017        Q1              REXR      USD  2017-03-31   2017-05-05   
            Q2              REXR      USD  2017-06-30   2017-08-04   
            Q3              REXR      USD  2017-09-30   2017-11-03   
            Q4              REXR      USD  2017-12-31   2018-02-21   
2018        Q1              REXR      USD  2018-03-31   2018-05-07   
            Q2              REXR      USD  2018-06-30   2018-08-06   
            Q3              REXR      USD  2018-09-30   2018-11-01   
            Q4              REXR      USD  2018-12-31   2019-02-19   

                           Shares (Basic)  Shares (Diluted)     Revenue  \
Fiscal Year Fiscal Period                                                 
2012        Q1                        NaN               NaN   7968000.0   
            Q2                        NaN               NaN   8234000.0   
2013        Q1                        NaN               NaN   9317000.0   
            Q2                        NaN               NaN  10822000.0   
            Q3                 24574432.0        24574432.0   8980000.0   
2014        Q1                 25419418.0        25419418.0  13691000.0   
            Q2                 25419757.0        25419757.0  14996000.0   
            Q3                 33527183.0        33527183.0  18036000.0   
            Q4                 43447666.0        43447666.0  19858000.0   
2015        Q1                 50683528.0        50683528.0  21340000.0   
            Q2                 54963093.0        54963093.0  22722000.0   
            Q3                 55145963.0        55145963.0  23674000.0   
            Q4                 55307108.0        55307108.0  26164000.0   
2016        Q1                 55269598.0        55416947.0  27504000.0   
            Q2                 64063337.0        64304713.0  30608000.0   
            Q3                 65707476.0        67985177.0  33303000.0   
            Q4                 65851673.0        64155379.0  34777000.0   
2017        Q1                 66341138.0        66626239.0  35354000.0   
            Q2                 67920773.0        68331234.0  36782000.0   
            Q3                 72621219.0        73068081.0  43339000.0   
            Q4                 77912318.0        78369062.0  45880000.0   
2018        Q1                 78694161.0        79196060.0  48536000.0   
            Q2                 82924208.0        83494825.0  51756000.0   
            Q3                 91463594.0        91945206.0  55194000.0   
            Q4                 94214977.0        94706905.0  57008000.0   

                           Cost of Revenue  Gross Profit  Operating Expenses  \
Fiscal Year Fiscal Period    

## Missing TTM Reports

Trailing-Twelve-Months (TTM) data is also quarterly so we can use the same helper-functions from above.

In [48]:
test_name = "Missing ttm reports"
test_func = lambda df: len(missing_quarterly_reports(df=df)) > 0
test_datasets(datasets=datasets_fundamental,
              variants=['ttm'],
              test_name=test_name, test_func=test_func)

Missing ttm reports
dataset='income', variant='ttm', market='de'
dataset='income', variant='ttm', market='sg'
dataset='income', variant='ttm', market='us'
dataset='income-insurance', variant='ttm', market='us'
dataset='balance', variant='ttm', market='de'
dataset='balance', variant='ttm', market='us'
dataset='balance-banks', variant='ttm', market='de'
dataset='balance-banks', variant='ttm', market='us'
dataset='balance-insurance', variant='ttm', market='us'
dataset='cashflow', variant='ttm', market='de'
dataset='cashflow', variant='ttm', market='us'
dataset='cashflow-banks', variant='ttm', market='de'
dataset='cashflow-banks', variant='ttm', market='us'
dataset='cashflow-insurance', variant='ttm', market='us'


Exception: Missing ttm reports

In [49]:
# Get list of SIMFIN_ID's that have missing reports for a dataset.
df = data.get(dataset='income', variant='ttm', market='us')
missing_quarterly_reports(df=df)

[67201,
 89750,
 117690,
 133865,
 181659,
 188922,
 233893,
 257294,
 258154,
 337852,
 340222,
 353395,
 370817,
 378142,
 410846,
 419436,
 436223,
 446592,
 515845,
 592569,
 609619,
 625536,
 626019,
 628033,
 630304,
 646016,
 651624,
 652699,
 653156,
 653270,
 654363,
 658373,
 658466,
 659172,
 660762,
 662524,
 662568,
 662704,
 679470,
 689588,
 703911,
 705165,
 705845,
 708961,
 708994,
 727266,
 728722,
 733746,
 767563,
 776098,
 778513,
 786011,
 798466,
 815635,
 819348,
 819902,
 823898,
 824090,
 824527,
 824887,
 825012,
 849042,
 877522,
 889944,
 890308,
 899359,
 901127,
 901866,
 905198,
 910562,
 919132,
 919501,
 923419,
 933165,
 942591,
 942592]

In [50]:
# Show all the reports for a given SIMFIN_ID sorted by
# Fiscal Year and Period so it is easier to see which are missing.
sort_quarterly_reports(df=df, simfin_id=89750)

Ticker Currency Report Date Publish Date  \
Fiscal Year Fiscal Period                                            
2008        Q4               DHR      USD  2008-12-31   2009-10-22   
2009        Q1               DHR      USD  2009-03-31   2010-04-22   
            Q2               DHR      USD  2009-06-30   2009-07-23   
            Q3               DHR      USD  2009-09-30   2009-10-22   
2010        Q4               DHR      USD  2010-12-31   2011-02-24   
2011        Q1               DHR      USD  2011-03-31   2011-04-21   
            Q2               DHR      USD  2011-06-30   2011-07-27   
            Q3               DHR      USD  2011-09-30   2011-10-20   
            Q4               DHR      USD  2011-12-31   2012-02-24   
2012        Q1               DHR      USD  2012-03-31   2012-04-19   
            Q2               DHR      USD  2012-06-30   2012-07-19   
            Q3               DHR      USD  2012-09-30   2012-10-18   
            Q4               DHR      USD  2012-12-31   2013-02-22   
2013        Q1               DHR      USD  2013-03-31   2013-04-18   
            Q2               DHR      USD  2013-06-30   2013-07-18   
            Q3               DHR      USD  2013-09-30   2013-10-17   
            Q4               DHR      USD  2013-12-31   2014-02-21   
2014        Q1               DHR      USD  2014-03-31   2014-04-16   
            Q2               DHR      USD  2014-06-30   2014-07-17   
            Q3               DHR      USD  2014-09-30   2014-10-16   
            Q4               DHR      USD  2014-12-31   2015-02-25   
2015        Q1               DHR      USD  2015-03-31   2015-04-23   
            Q2               DHR      USD  2015-06-30   2015-07-23   
            Q3               DHR      USD  2015-09-30   2015-10-22   
            Q4               DHR      USD  2015-12-31   2016-02-24   
2016        Q1               DHR      USD  2016-03-31   2016-04-21   
            Q2               DHR      USD  2016-06-30   2016-07-25   
            Q3               DHR      USD  2016-09-30   2016-10-20   
            Q4               DHR      USD  2016-12-31   2017-02-22   
2017        Q1               DHR      USD  2017-03-31   2017-04-20   
            Q2               DHR      USD  2017-06-30   2017-07-20   
            Q3               DHR      USD  2017-09-30   2017-10-19   
            Q4               DHR      USD  2017-12-31   2018-02-21   
2018        Q1               DHR      USD  2018-03-31   2018-04-19   
            Q2               DHR      USD  2018-06-30   2018-07-19   
            Q3               DHR      USD  2018-09-30   2018-10-18   
            Q4               DHR      USD  2018-12-31   2019-02-21   
2019        Q1               DHR      USD  2019-03-31   2019-04-18   

                           Shares (Basic)  Shares (Diluted)       Revenue  \
Fiscal Year Fiscal Period                                                   
2008        Q4                843114360.0       886680300.0  1.269746e+10   
2009        Q1                843466140.0       885034260.0  1.229633e+10   
            Q2                843916920.0       883707000.0  1.168604e+10   
            Q3                844712880.0       883003440.0  1.122855e+10   
2010        Q4                862224000.0       901956000.0  1.255000e+10   
2011        Q1                866392230.0       904663980.0  1.283437e+10   
            Q2                871252800.0       908710110.0  1.325281e+10   
            Q3                882030600.0       917610870.0  1.460576e+10   
            Q4                892584000.0       925584000.0  1.609050e+10   
2012        Q1                902451330.0       934055760.0  1.711450e+10   
            Q2                911821020.0       940755090.0  1.803213e+10   
            Q3                914441550.0       941909100.0  1.800184e+10   
            Q4                915288000.0       941292000.0  1.826040e+10   
2013        Q1                915453000.0       939444000.0  1.838890e